# 01 — Explore backscatter & change detection (VV vs VH)
This notebook helps you **inspect SAR backscatter change** and **pick thresholds** using histograms.

It expects you already ran `sar101.py` and produced outputs.

## Recommended runs (to compare VV vs VH)
From the repo root:
```bash
python src/sar101.py --prefer-polarization vv --outdir outputs_vv
python src/sar101.py --prefer-polarization vh --outdir outputs_vh
```
Each run should create `ratio_db_t1_minus_t0.tif` (and quicklooks).


In [ ]:
from pathlib import Path
import numpy as np
import rasterio
import matplotlib.pyplot as plt

# Update these paths if needed
VV_DIR = Path('../outputs_vv')
VH_DIR = Path('../outputs_vh')

vv_ratio_path = VV_DIR / 'ratio_db_t1_minus_t0.tif'
vh_ratio_path = VH_DIR / 'ratio_db_t1_minus_t0.tif'

print('VV ratio:', vv_ratio_path, 'exists=', vv_ratio_path.exists())
print('VH ratio:', vh_ratio_path, 'exists=', vh_ratio_path.exists())


## Load rasters
We load the **ratio in dB**: `ratio_db = db(t1) - db(t0)`.
Positive values = backscatter increased; negative = decreased.


In [ ]:
def read_band_as_array(path: Path):
    with rasterio.open(path) as ds:
        arr = ds.read(1).astype('float32')
        nodata = ds.nodata
        if nodata is not None:
            arr[arr == nodata] = np.nan
        return arr, ds

vv_ratio, vv_ds = read_band_as_array(vv_ratio_path)
vh_ratio, vh_ds = read_band_as_array(vh_ratio_path)

print('VV ratio stats:', np.nanmin(vv_ratio), np.nanmean(vv_ratio), np.nanmax(vv_ratio))
print('VH ratio stats:', np.nanmin(vh_ratio), np.nanmean(vh_ratio), np.nanmax(vh_ratio))


## Histograms
Use histograms to pick a **change threshold** (`--change-thr-db`).
A common starting point is **1–3 dB**, but it depends on terrain, incidence, and speckle.


In [ ]:
def plot_hist(arr, title, bins=200, xlim=(-10, 10)):
    data = arr[np.isfinite(arr)]
    plt.figure()
    plt.hist(data, bins=bins)
    plt.title(title)
    plt.xlabel('ratio_db = db(t1) - db(t0)')
    plt.ylabel('pixel count')
    plt.xlim(*xlim)
    plt.grid(True, alpha=0.3)
    plt.show()

plot_hist(vv_ratio, 'VV ratio_db histogram')
plot_hist(vh_ratio, 'VH ratio_db histogram')


## Visual comparison (quick maps)
These are raw pixel maps. Look for:
- bright areas (large positive ratio)
- dark areas (large negative ratio)
- speckle (salt-and-pepper)


In [ ]:
def plot_map(arr, title, vmin=-5, vmax=5):
    plt.figure(figsize=(7, 6))
    plt.imshow(arr, vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.colorbar(label='dB')
    plt.axis('off')
    plt.show()

plot_map(vv_ratio, 'VV ratio_db (db(t1)-db(t0))')
plot_map(vh_ratio, 'VH ratio_db (db(t1)-db(t0))')


## Pick a threshold visually
Try a few thresholds and see how the **change mask** behaves.
This matches the script logic: `abs(ratio_db) > change_thr_db`.


In [ ]:
def mask_from_threshold(arr, thr_db):
    return (np.abs(arr) > thr_db).astype('uint8')

for thr in [1.0, 1.5, 2.0, 3.0]:
    m = mask_from_threshold(vv_ratio, thr)
    pct = 100.0 * m.mean()
    print(f'VV change_thr_db={thr:.1f} -> {pct:.2f}% pixels flagged')

thr_db = 2.0
vv_mask = mask_from_threshold(vv_ratio, thr_db)
vh_mask = mask_from_threshold(vh_ratio, thr_db)
plot_map(vv_mask, f'VV change mask (thr={thr_db} dB)', vmin=0, vmax=1)
plot_map(vh_mask, f'VH change mask (thr={thr_db} dB)', vmin=0, vmax=1)


## Notes on VV vs VH
- **VV** often highlights man-made structure changes and water surfaces.
- **VH** is often more sensitive to vegetation structure and volume scattering.
Use both if you can, but start with VV for general situational awareness.
